In [3]:
import pandas as pd

df_lung = pd.read_csv("hcp_marketing_metrics_100k.csv")
df_lung.head()

,bp_id,month,brand,indication,vendor,vehicle,metric_category,value
0,HCP_00861,2024-10,BrandA,Oncology,VendorX,Email,REACH,4
1,HCP_03773,2024-02,BrandB,Cardiology,VendorZ,Rep Visit,ENGAGEMENT,1
2,HCP_03093,2024-02,BrandC,Diabetes,VendorY,WhatsApp,REACH,3
3,HCP_00467,2024-03,BrandA,Cardiology,VendorX,WhatsApp,ENGAGEMENT,2
4,HCP_04427,2024-06,BrandC,Cardiology,VendorZ,Email,ENGAGEMENT,1


In [7]:
import pandas as pd

def calculate_metrics(df, group_by_cols):
    reach_df = df[df["metric_category"] == "REACH"]
    engagement_df = df[df["metric_category"] == "ENGAGEMENT"]

    result = (
        reach_df
        .groupby(group_by_cols)
        .agg(
            HCP_REACHED=("bp_id", "nunique"),
            REACHED_TPS=("value", "sum")
        )
        .reset_index()
    )

    engaged = (
        engagement_df
        .groupby(group_by_cols)
        .agg(
            HCPS_ENGAGED=("bp_id", "nunique"),
            ENGAGED_TPS=("value", "sum")
        )
        .reset_index()
    )

    final = result.merge(engaged, on=group_by_cols, how="left")
    final["HCPS_ENGAGED"] = final["HCPS_ENGAGED"].fillna(0)
    final["ENGAGED_TPS"] = final["ENGAGED_TPS"].fillna(0)

    return final


In [32]:
from itertools import combinations

DIMENSIONS = [
    "brand",
    "indication",
    "vendor",
    "month",
    "vehicle"
]

def build_metric_cube(df):
    cubes = {}

    for i in range(1, len(DIMENSIONS) + 1):
        for combo in combinations(sorted(DIMENSIONS), i):
            key = tuple(sorted(combo))   # canonical key

            cubes[key] = calculate_metrics(
                df=df,
                group_by_cols=list(combo)
            )

    return cubes


In [33]:
metric_cube = build_metric_cube(df_lung)

def answer_question(granularity):
    key = tuple(sorted(granularity))
    return metric_cube.get(key, "Granularity not supported")


In [37]:
answer_question([ "brand","month"])

,brand,month,HCP_REACHED,REACHED_TPS,HCPS_ENGAGED,ENGAGED_TPS
0,BrandA,2024-01,1015,2854,999,1172
1,BrandA,2024-02,999,2769,1040,1148
2,BrandA,2024-03,1002,2815,988,1138
3,BrandA,2024-04,1000,2724,961,1078
4,BrandA,2024-05,1075,2943,1012,1138
5,BrandA,2024-06,994,2772,942,1017
6,BrandA,2024-07,938,2653,954,1069
7,BrandA,2024-08,980,2763,1022,1141
8,BrandA,2024-09,932,2516,1007,1093
9,BrandA,2024-10,1016,2860,1008,1165


In [39]:
import streamlit as st

DIMENSIONS = [
    "vendor",
    "month",
    "vehicle",
    "brand",
    "indicstion"
]

st.set_page_config(page_title="Metric Cube Explorer", layout="wide")

st.title("📊 Metric Cube Explorer")

st.markdown("Select any combination of dimensions to view the summary metrics.")

# --- Dimension selector ---
selected_dims = st.multiselect(
    "Choose dimensions",
    DIMENSIONS
)

# --- Fetch result ---
if selected_dims:
    key = tuple(sorted(selected_dims))
    result = metric_cube.get(key)

    if result is None:
        st.warning("Granularity not supported")
    else:
        st.success(f"Showing metrics for: {', '.join(key)}")
        st.dataframe(result, use_container_width=True)

        # Optional download
        csv = result.to_csv(index=False)
        st.download_button(
            "Download CSV",
            csv,
            file_name=f"metrics_{'_'.join(key)}.csv",
            mime="text/csv"
        )
else:
    st.info("Please select at least one dimension.")


2026-02-11 01:46:10.123 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 01:46:10.124 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 01:46:10.350 
  command:

    streamlit run C:\Users\ani12\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2026-02-11 01:46:10.351 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 01:46:10.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 01:46:10.353 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 01:46:10.353 Thread 'MainThread': missing ScriptRunContext! This wa